In [14]:
# # Document loading, retrieval methods and text splitting
# Follow readme to install virtual env and packages.

# # Pull the model first
# !ollama pull nomic-embed-text



In [15]:
#Imports
import os
import gradio as gr
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import ChatOllama

In [16]:
# Load Company Files
loader = DirectoryLoader("../../company-files/", glob="./*.txt",   loader_cls=TextLoader)
# Load the document from the TXT file
documents = loader.load()

# Print the loaded document
print(documents)

[Document(metadata={'source': '../../company-files/shipping.txt'}, page_content='# TechStyle Global - Shipping Policy\n\n## Delivery Options\n\n### Standard Shipping\n- Delivery Time: 3-5 business days\n- Cost: Free for orders over $50\n- Orders under $50: $4.99\n- Available in: United States and Canada\n- Tracking provided via email\n\n### Express Shipping\n- Delivery Time: 1-2 business days\n- Cost: $9.99\n- Free for TechStyle Plus members\n- Available in: United States and Canada\n- Real-time tracking via app and email\n\n### International Shipping\n- Delivery Time: 7-14 business days\n- Cost: Calculated based on:\n  - Destination country\n  - Package weight\n  - Package dimensions\n- Available in: 50+ countries\n- Tracking provided where available\n\n## Order Processing\n\n### Processing Times\n- In-stock items: 1-2 business days\n- Custom orders: 3-5 business days\n- Pre-orders: As specified on product page\n- Business days are Monday-Friday, excluding holidays\n\n### Cut-off Time

In [18]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [83]:
# Put the chunks of data into a Vector Store that associates a Vector Embedding with each chunk
# Chroma is a popular open source Vector Database based on SQLLite
DB_NAME = "vector_db"

embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Delete if already exists

if os.path.exists(DB_NAME):
    Chroma(persist_directory=DB_NAME, embedding_function=embeddings).delete_collection()

# Create vectorstore

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=DB_NAME)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 13 documents


In [20]:
#run a quick test - should return a list of documents = 4
question = "When was TechStyle Global founded?"
docs = vectorstore.similarity_search(question)
len(docs)

4

In [82]:
# create a new Chat with Ollama
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
MODEL = "llama3.2:latest"
llm = ChatOllama(temperature=0.7, model=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [43]:
# Let's try a simple question

query = "What is the price of Premium Screen Protector Set?"
result = conversation_chain.invoke({"question": query})
print(result["answer"])

The price of the Premium Screen Protector Set is $19.99.


In [45]:
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [46]:
# And in Gradio:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


### Conversational memory using Langgraph

In [133]:
# LangGraph for conversational memory as ConversationBufferMemory is deprecated
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langchain_core.messages import HumanMessage
from langchain.tools.retriever import create_retriever_tool
from langgraph.prebuilt import ToolNode, tools_condition
import uuid
# Define a new graph
graph_builder = StateGraph(state_schema=MessagesState)

# Define  ollama chat model
OLLAMA_MODEL = "llama3.2:latest"
llm_model = ChatOllama(temperature=0.7, model=OLLAMA_MODEL)


# Create retrieval tool to read from vector store
retriever_tool = create_retriever_tool(
    vectorstore.as_retriever(),
    "retrieve_my_texts",
    "Retrieve company texts stored vector store",
)
tools = [retriever_tool]
llm_with_tools = llm_model.bind_tools(tools)

# Define the function that calls the model
def chatbot_llm_tools(state: MessagesState):
    response = llm_with_tools.invoke(state["messages"])
    # We return a list, because this will get added to the existing list
    return {"messages": response}

# Define the two nodes we will cycle between
graph_builder.add_node("chatbot",chatbot_llm_tools)
tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# Adding memory is straight forward in langgraph!
memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)

# The thread id is a unique key that identifies
# this particular conversation.
# We'll just generate a random uuid here.
# This enables a single application to manage conversations among multiple users.
thread_id = uuid.uuid4()
config = {"configurable": {"thread_id": thread_id}}

def chatbot_langgraph(user_input):
    response = graph.invoke({
        "messages": [HumanMessage(content=user_input)]
    }, config)
    # Format the response if needed
    formatted_response = response['messages'][-1].content
    formatted_response = formatted_response.replace("\\n", "\n")
    return formatted_response

In [134]:
chatbot_langgraph("what is the price of Premium Screen Protector Set?")


'The price of the Premium Screen Protector Set is $19.99.'

In [136]:
chatbot_langgraph("what is the presence of Tech Style Global?")


'TechStyle Global is a leading e-commerce marketplace specializing in consumer electronics, fashion, and lifestyle products. The company was founded in 2015 by tech entrepreneurs Sarah Chen and Marcus Rodriguez with the vision of creating a seamless shopping experience that bridges technology and lifestyle products.\n\nThe presence of Tech Style Global can be seen on various online platforms, including their official website, social media channels, and marketplaces like Amazon. They offer a wide range of products, from smartphones and laptops to smart home devices and fashion accessories.\n\nAs of my knowledge cutoff in 2023, TechStyle Global operates globally with shipping options available in over 50 countries. Their delivery times vary depending on the destination country and product type, but they provide tracking information via email and app for express shipments.'

In [135]:
lg_view = gr.ChatInterface(chatbot_langgraph, type="messages").launch(inbrowser=True)


* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.
